### brunch 작가 데이터 수집 
1. 브런치 follower, following 정보 수집 
1. 브런치 작기 정보 수집

In [20]:
import pandas as pd
import matplotlib.pylab as plt
import requests
import json
from bs4 import BeautifulSoup
%matplotlib inline

In [4]:
## road pickle data
import pickle
pkl_file = open('/Users/goodvc/Data/fc-recsys-school/ch1/resource/brunch_sns.pkl', 'rb')
ds_dict = pickle.load(pkl_file)
pkl_file.close() 

In [31]:
## 관심 작가 정보 크롤링 하기 
## extract Brunch Writer Info : uid, name, text-count, megazine-count, follower-count, following-count:
def extractWriterInfo(uid):
    try:
        response = requests.get("http://brunch.co.kr/@{uid}".format(uid=uid) )
    except Exception:
        []
        
    data = response.content.decode('utf-8')
    soup = BeautifulSoup(data, 'html.parser')
    
    ## name 
    names = soup.find_all("strong", class_="profileUserName")
    name = uid if len(names)<1 else names[0].getText()
    
    classes = soup.find_all("span", class_="num_count")
    reserved = [uid, name]
    for c in classes:
        reserved.append(int(c.getText().replace(',','')))
    return reserved

print extractWriterInfo('goodvc78')

['goodvc78', u'\ucd5c\uaddc\ubbfc', 3, 1, 158, 8]


In [14]:
def toDataframe(src) :
    data = []
    for n, f in src.items():
        for nn in f['following']:
            data.append([n,'following',nn])
    df = pd.DataFrame(data, columns=['uid','dir','target'])
    return df[~df.target.isin(['brunch'])]

In [41]:
df = toDataframe(ds_dict['goodvc78'])
df = df[~df.target.isin(['goodvc78','brunch'])]

158

### 상위 브런치 작가

In [67]:
## 상위 50 작가 선정 
topn=50
top_writer = df[df.dir=='following'].groupby('target').count().sort('uid', ascending=False)[:topn][['dir']]

top_writer = top_writer.reset_index()
top_writer.columns = ['uid','교차 구독자수']

## 상위 작가의 bruch 정보 가져오기 ( 크롤링 ) 
top_writer_info = []
for uid in top_writer.uid:
    top_writer_info.append(extractWriterInfo(uid))
top_writer_info = pd.DataFrame(top_writer_info, columns=['uid','작가명','글 수', '매거진수', '비교작가 구독자수', '관심작가수' ])

## 데이터 머지 
top_writer = pd.merge(top_writer, top_writer_info, on=['uid'])
top_writer['관심작가 구독자수'] = len(ds_dict['goodvc78'].keys())

## 전체 구독수대비 나의 구독자수 비율
top_writer['교차 구독자수 비율'] = top_writer['교차 구독자수'] / (top_writer['비교작가 구독자수'] + top_writer['관심작가 구독자수'] - top_writer['교차 구독자수']  )
top_writer['교차 구독자수 비율'] = top_writer['교차 구독자수 비율'].apply(lambda x: round(x,3))

In [68]:
top_writer.head()

,uid,교차 구독자수,작가명,글 수,매거진수,비교작가 구독자수,관심작가수,관심작가 구독자수,교차 구독자수 비율
0,lifidea,54,Jin Young Kim,10,1,786,194,158,0.061
1,brunchflgu,40,조우성 변호사,674,10,5243,35,158,0.007
2,jimmyrim,40,임지훈 Jimmy Rim,6,0,2469,0,158,0.015
3,yoojs8512,38,유재석,46,1,1468,24,158,0.024
4,insuk,35,조인석 chris,10,1,939,29,158,0.033


In [69]:
def rankBy( ds, colname, topn=10 ):
    ranked = ds.sort(colname, ascending=False)[['uid','작가명','관심작가 구독자수', '교차 구독자수','비교작가 구독자수', '교차 구독자수 비율']][:topn]
    for (name,row) in ranked.iterrows():
        print ("https://brunch.co.kr/@{name}".format(name=row['uid'], value=row['작가명']))
    return ranked
#rankBy( top_writer, '나의 구독자수')

In [70]:
rankBy( top_writer, '교차 구독자수')

https://brunch.co.kr/@lifidea
https://brunch.co.kr/@jimmyrim
https://brunch.co.kr/@brunchflgu
https://brunch.co.kr/@yoojs8512
https://brunch.co.kr/@insuk
https://brunch.co.kr/@suhanjang
https://brunch.co.kr/@suyoung
https://brunch.co.kr/@pelexus
https://brunch.co.kr/@haneulalice
https://brunch.co.kr/@wjchee


,uid,작가명,관심작가 구독자수,교차 구독자수,비교작가 구독자수,교차 구독자수 비율
0,lifidea,Jin Young Kim,158,54,786,0.061
2,jimmyrim,임지훈 Jimmy Rim,158,40,2469,0.015
1,brunchflgu,조우성 변호사,158,40,5243,0.007
3,yoojs8512,유재석,158,38,1468,0.024
4,insuk,조인석 chris,158,35,939,0.033
5,suhanjang,티거 Jang,158,31,4965,0.006
6,suyoung,강수영,158,31,1265,0.022
7,pelexus,최윤섭,158,27,819,0.028
8,haneulalice,Alice in wonderland,158,25,2570,0.009
9,wjchee,지원준,158,25,468,0.042


In [73]:
rankBy( top_writer, '비교작가 구독자수')

https://brunch.co.kr/@brunchflgu
https://brunch.co.kr/@suhanjang
https://brunch.co.kr/@captaink
https://brunch.co.kr/@rothem
https://brunch.co.kr/@haneulalice
https://brunch.co.kr/@jimmyrim
https://brunch.co.kr/@lonelyplanet
https://brunch.co.kr/@2kija
https://brunch.co.kr/@sooscape
https://brunch.co.kr/@yoonash


,uid,작가명,관심작가 구독자수,교차 구독자수,비교작가 구독자수,교차 구독자수 비율
1,brunchflgu,조우성 변호사,158,40,5243,0.007
5,suhanjang,티거 Jang,158,31,4965,0.006
46,captaink,캡틴K,158,13,3907,0.003
38,rothem,Rothem,158,15,2695,0.005
8,haneulalice,Alice in wonderland,158,25,2570,0.009
2,jimmyrim,임지훈 Jimmy Rim,158,40,2469,0.015
49,lonelyplanet,론리플래닛 매거진 코리아,158,13,2292,0.005
33,2kija,이기주작가,158,16,2234,0.007
12,sooscape,흔디,158,24,1758,0.013
13,yoonash,yoonash,158,24,1478,0.015


In [74]:
rankBy( top_writer, '교차 구독자수 비율')

https://brunch.co.kr/@cojette
https://brunch.co.kr/@lifidea
https://brunch.co.kr/@chaibschaibs
https://brunch.co.kr/@cloud09
https://brunch.co.kr/@yannheo
https://brunch.co.kr/@manya
https://brunch.co.kr/@wjchee
https://brunch.co.kr/@genie7pe
https://brunch.co.kr/@jaeseungmun
https://brunch.co.kr/@yper


,uid,작가명,관심작가 구독자수,교차 구독자수,비교작가 구독자수,교차 구독자수 비율
19,cojette,cojette,158,21,95,0.091
0,lifidea,Jin Young Kim,158,54,786,0.061
34,chaibschaibs,CHAIBS,158,16,120,0.061
32,cloud09,강한별,158,16,170,0.051
21,yannheo,허양일,158,19,290,0.044
17,manya,마냐,158,21,341,0.044
9,wjchee,지원준,158,25,468,0.042
40,genie7pe,Chulhyun Cho,158,14,198,0.041
42,jaeseungmun,Jaeseung Mun,158,14,210,0.040
29,yper,문현구 YPER 대표,158,17,308,0.038
